### Mapping state-level changes to real GDP

Data source: U.S. Bureau of Economic Analysis

https://www.bea.gov/newsreleases/regional/gdp_state/qgsp_newsrelease.htm



In [13]:
import requests
import pandas as pd
import config   ## File with API key
api_key = config.key

# Components of request
base = 'https://www.bea.gov/api/data/?&UserID={}'.format(api_key)
m = '&method=GetData'
dsn = '&datasetname=RegionalProduct'
ind = '&IndustryId=1'
comp = '&Component=RGDP_SQN'
geo = '&GeoFIPS=STATE'
year = '&Year=LAST5'
fmt = '&ResultFormat=json'

# Combined url for request
url = '{}{}{}{}{}{}{}{}'.format(base, m, dsn, ind, comp, geo, year, fmt)

# Load list of state names and postal abbreviations
states = pd.read_csv('state_abbrevs.csv').set_index('State')

In [14]:
# Make request and load data into pandas dataframe, show last 5 obs
r = requests.get(url).json()
df = pd.DataFrame(r['BEAAPI']['Results']['Data'])
df = df[df['GeoName'].isin(states.index)]
df['date'] = pd.to_datetime(df['TimePeriod'])
df = df.sort_values(['GeoName', 'date'])
df = df.set_index(['GeoName', 'date'])['DataValue']
df = df.apply(pd.to_numeric, errors='coerce').unstack(0)

In [15]:
# Calculate growth rate by state
for s in df.keys():
    df[s] = ((1 + (df[s] - df[s].shift(1))/df[s].shift(1))**4 - 1) * 100

In [16]:
date = '{} Q{}'.format(df.index[-1].year, df.index[-1].quarter)
df1 = df.tail(1).stack().reset_index()[['GeoName', 0]]
df1['Abbrev'] = df1['GeoName'].map(states.to_dict()['Abbreviation'])
df1 = df1.set_index('GeoName').set_value('Michigan', 'Abbrev', 'MI, SP')

#### Convert to .tex input



In [17]:
# Tex strings 
c_str = '\\tikzset{set state val/.style args={#1}{#1='
s_str = '\\tikzset{set state val/.list={'

# Color categories
C1 = '{fill=blue!50!green}}}'
C2 = '{fill=green!80!blue}}}'
C3 = '{fill=yellow!70!green}}}'
C4 = '{fill=orange!60!yellow}}}'
C5 = '{fill=red!40!orange!80}}}'
C6 = '{fill=red!80!black!80}}}'

In [18]:
S1 = ', '.join(map(str, df1[df1[0] >= 5]['Abbrev'].values))
S2 = ', '.join(map(str, df1[(df1[0] < 5) & (df1[0] >= 2)]['Abbrev'].values))
S3 = ', '.join(map(str, df1[(df1[0] < 2) & (df1[0] >= 0)]['Abbrev'].values))
S4 = ', '.join(map(str, df1[(df1[0] < 0) & (df1[0] >= -2)]['Abbrev'].values))
S5 = ', '.join(map(str, df1[(df1[0] < -2) & (df1[0] >= -5)]['Abbrev'].values))
S6 = ', '.join(map(str, df1[df1[0] < -5]['Abbrev'].values))

In [19]:
d = {C1: S1, C2: S2, C3: S3, C4: S4, C5: S5, C6: S6}
for key, value in d.iteritems():
    print '{}{}\n{}{}}}}}'.format(c_str, key, s_str, value)

\tikzset{set state val/.style args={#1}{#1={fill=red!80!black!80}}}
\tikzset{set state val/.list={}}
\tikzset{set state val/.style args={#1}{#1={fill=yellow!70!green}}}
\tikzset{set state val/.list={AK, AZ, AR, CO, DE, GA, HI, IL, IN, IA, KS, KY, LA, ME, MD, MN, MS, MO, MT, NE, NV, NJ, NM, NY, NC, ND, OH, OK, PA, RI, SC, SD, TN, VT, VA, WV, WI, WY}}
\tikzset{set state val/.style args={#1}{#1={fill=green!80!blue}}}
\tikzset{set state val/.list={AL, CA, CT, DC, FL, ID, MA, MI, SP, NH, OR, TX, UT, WA}}
\tikzset{set state val/.style args={#1}{#1={fill=red!40!orange!80}}}
\tikzset{set state val/.list={}}
\tikzset{set state val/.style args={#1}{#1={fill=blue!50!green}}}
\tikzset{set state val/.list={}}
\tikzset{set state val/.style args={#1}{#1={fill=orange!60!yellow}}}
\tikzset{set state val/.list={}}


In [20]:
df1.sort_values(0)

,0,Abbrev
GeoName,,
Mississippi,0.050419,MS
Kansas,0.064234,KS
Wyoming,0.223658,WY
Iowa,0.375990,IA
New York,0.445107,NY
Arkansas,0.450279,AR
Alaska,0.519452,AK
Maine,0.679862,ME
Indiana,0.787464,IN


In [22]:
date

'2016 Q4'